In [1]:
!git clone https://github.com/altaga/Facemask-Opt-Dataset

Cloning into 'Facemask-Opt-Dataset'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 2048 (delta 3), reused 48 (delta 3), pack-reused 1998
Receiving objects: 100% (2048/2048), 109.06 MiB | 30.96 MiB/s, done.
Resolving deltas: 100% (4/4), done.
Updating files: 100% (1901/1901), done.


In [2]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import azureml
from azureml.core import Workspace, Run
ws = Workspace.from_config()

experiment_name = 'deeplearning_facemask'
from azureml.core import Experiment
exp = Experiment(workspace=ws, name=experiment_name)
print(exp)

Experiment(Name: deeplearning_facemask,
Workspace: azureai)


In [3]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

# choose a name for your cluster
compute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", "cpu-cluster")
compute_min_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES", 0)
compute_max_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES", 4)

# This example uses CPU VM. For using GPU VM, set SKU to STANDARD_NC6
vm_size = os.environ.get("AML_COMPUTE_CLUSTER_SKU", "STANDARD_D2_V2")


if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print("found compute target: " + compute_name)
else:
    print("creating new compute target...")
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
                                                                min_nodes = compute_min_nodes, 
                                                                max_nodes = compute_max_nodes)

    # create the cluster
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)
    
    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
     # For a more detailed view of current AmlCompute status, use get_status()
    print(compute_target.get_status().serialize())

found compute target: cpu-cluster


In [4]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

# to install required packages
env = Environment('facemaskenv')
cd = CondaDependencies.create(pip_packages=['Pillow','azureml-defaults',"tensorflow","scikit-learn","opencv-python-headless"])
env.python.conda_dependencies = cd

# Register environment to re-use later
env.register(workspace = ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/intelmpi2018.3-ubuntu16.04:20210104.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "facemaskenv",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-forge"


In [5]:
from azureml.core import ScriptRunConfig

src = ScriptRunConfig(source_directory='',
                compute_target=compute_target,
                script='train.py',
                environment=env)

In [6]:
run = Experiment(workspace=ws, name='TF-Facemask').submit(src)
run.wait_for_completion(show_output=True)

Submitting /mnt/batch/tasks/shared/LS_root/mounts/clusters/azureaitrain/code/Users/victor.altamirano directory for run. The size of the directory >= 25 MB, so it can take a few minutes.


RunId: TF-Facemask_1615160023_257c1600
Web View: https://ml.azure.com/experiments/TF-Facemask/runs/TF-Facemask_1615160023_257c1600?wsid=/subscriptions/e75c3056-1282-45ed-b69c-56d4bec0fa9e/resourcegroups/azureai/workspaces/azureai

Streaming azureml-logs/55_azureml-execution-tvmps_44b6faadcdf1614c287768b2145a0d1a3d41692a0352e31bdff50356c4ca9bd4_d.txt

2021-03-07T23:42:15Z Starting output-watcher...
2021-03-07T23:42:15Z IsDedicatedCompute == True, won't poll for Low Pri Preemption
2021-03-07T23:42:16Z Executing 'Copy ACR Details file' on 10.0.0.4
2021-03-07T23:42:16Z Copy ACR Details file succeeded on 10.0.0.4. Output: 
>>>   
>>>   
Login Succeeded
Using default tag: latest
latest: Pulling from azureml/azureml_694057430b94b82fc18360ae5a473464
be8ec4e48d7f: Pulling fs layer
33b8b485aff0: Pulling fs layer
d887158cc58c: Pulling fs layer
05895bb28c18: Pulling fs layer
baf7ab26f516: Pulling fs layer
181182e3c9cf: Pulling fs layer
d584ef274e55: Pulling fs layer
c445dda55407: Pulling fs layer


{'runId': 'TF-Facemask_1615160023_257c1600',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-03-07T23:42:17.18757Z',
 'endTimeUtc': '2021-03-07T23:48:48.073467Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': 'b3610190-6565-4795-9c6a-cc4546f56871',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'train.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': [],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'cpu-cluster',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'jobName': None,
  'maxRunDurationSeconds': 2592000,
  'nodeCount': 1,
  'priority': None,
  'credentialPassthrough': False,
  'identity': None,
  'environment': {'name': 'facemaskenv',
   'version': '4',
   'python': {'interpreterPath': 'python',
  